In [5]:
import os

import numpy as np
import pandas as pd
import xlrd

In [26]:
data_path = '/Users/user/Documents/Pentatonic/Travelers/data'

files = [os.path.join(data_path, f) for f in os.listdir(data_path) if os.path.isfile(os.path.join(data_path, f)) and "Xact_Categories" in f and f.endswith(".xlsx")]

files

['/Users/user/Documents/Pentatonic/Travelers/data/Pentatonic_Xact_Categories_5yr_Final_5.xlsx',
 '/Users/user/Documents/Pentatonic/Travelers/data/Pentatonic_Xact_Categories_5yr_Final_4.xlsx',
 '/Users/user/Documents/Pentatonic/Travelers/data/Pentatonic_Xact_Categories_5yr_Final_3.xlsx',
 '/Users/user/Documents/Pentatonic/Travelers/data/Pentatonic_Xact_Categories_5yr_Final_2.xlsx',
 '/Users/user/Documents/Pentatonic/Travelers/data/Pentatonic_Xact_Categories_5yr_Final_1.xlsx',
 '/Users/user/Documents/Pentatonic/Travelers/data/Pentatonic_Xact_Categories_5yr_Final_6.xlsx']

In [28]:
temp_data = []
for file in files:
    temp_data.append(pd.read_excel(file))
    
data = pd.concat(temp_data, ignore_index=True)

In [29]:
data.shape[0]

1357348

In [47]:
rename_map = {
    "category_description" : "subcategory",
    "ITM_TYP_DESC_2" : "item_description",
    "DIV_CD" : "div_cd",
    "LS_YYMM" : "ls_date",
    "peril_grp" : "reason",
    "ITM_UNIT_CD" : "item_unit_cd",
    "ITM_QTY" : "item_quantity",
    "count1" : "count"
}

data = data.rename(columns=rename_map)

In [48]:
# data.drop(columns=["primary_col_desc"], inplace=True)

In [49]:
grouped = data.groupby('subcategory')
categories = list(grouped.groups)
print("Number of categories: {}".format(len(categories)))
print("Categories: {}".format(categories))

Number of categories: 121
Categories: ['ACCESSORIES - MOBILE HOME', 'ACOUSTICAL TREATMENTS', 'ANTIQUES & VINTAGE GOODS', 'APPLIANCES', 'APPLIANCES - MAJOR W/O INSTALL', 'APPLIANCES - SMALL', 'ART RESTORATION, CONSERVATION', 'ARTWORK', 'AUTOMOTIVE & MOTORCYCLE ACC.', 'AWNINGS & PATIO COVERS', 'BOOKS, MAGAZINES & PERIODICALS', 'BUSINESS GOODS & EQUIPMENT', 'CABINETRY', 'CAMERAS, CAMCORDERS & EQUIP.', 'CASH & SECURITIES', 'CLEANING', 'CLOTHING & ACCESSORIES', 'COMPUTERS & RELATED GOODS', 'CONCRETE & ASPHALT', 'CONT: CEILING/WALL HANGINGS', 'CONT: CLEAN - GENERAL ITEMS', 'CONT: CLEAN - HARD FURNITURE', 'CONT: CLEAN - LAMPS OR VASES', 'CONT: CLEAN APPLIANCES', 'CONT: CLEAN ELECTRIC ITEMS', 'CONT: CLEAN,UPHOLSTRY & SOFT', 'CONT: GARMENT & SOFT GOODS CLN', 'CONT: PACKING,HANDLNG,STORAGE', 'CONTENT MANIPULATION', 'CREDIT', 'DOCUMENTS & VALUABLE PAPERS', 'DOORS', 'DRYWALL', 'ELECTRICAL', 'ELECTRICAL - SPECIAL SYSTEMS', 'ELECTRONICS', 'EXCAVATION', 'EXTERIOR STRUCTURES', 'FEES - CONTENTS MISC.',

Subcategories: [<font color='green'>'ACCESSORIES - MOBILE HOME'</font>, <font color='green'>'ACOUSTICAL TREATMENTS'</font>, 'ANTIQUES & VINTAGE GOODS', 'APPLIANCES', 'APPLIANCES - MAJOR W/O INSTALL', 'APPLIANCES - SMALL', 'ART RESTORATION, CONSERVATION', 'ARTWORK', 'AUTOMOTIVE & MOTORCYCLE ACC.', 'AWNINGS & PATIO COVERS', <font color='green'>'BOOKS, MAGAZINES & PERIODICALS'</font>, 'BUSINESS GOODS & EQUIPMENT', <font color='green'>'CABINETRY'</font>, <font color='green'>'CAMERAS, CAMCORDERS & EQUIP.'</font>, 'CASH & SECURITIES', 'CLEANING', <font color='green'>'CLOTHING & ACCESSORIES'</font>, <font color='green'>'COMPUTERS & RELATED GOODS'</font>, <font color='green'>'CONCRETE & ASPHALT'</font>, <font color='green'>'CONT: CEILING/WALL HANGINGS'</font>, 'CONT: CLEAN - GENERAL ITEMS', <font color='green'>'CONT: CLEAN - HARD FURNITURE'</font>, <font color='green'>'CONT: CLEAN - LAMPS OR VASES'</font>, 'CONT: CLEAN APPLIANCES', <font color='green'>'CONT: CLEAN ELECTRIC ITEMS'</font>, <font color='green'>'CONT: CLEAN,UPHOLSTRY & SOFT'</font>, <font color='green'>'CONT: GARMENT & SOFT GOODS CLN'</font>, 'CONT: PACKING,HANDLNG,STORAGE', 'CONTENT MANIPULATION', 'CREDIT', <font color='green'>'DOCUMENTS & VALUABLE PAPERS'</font>, <font color='green'>'DOORS'</font>, <font color='green'>'DRYWALL'</font>, <font color='green'>'ELECTRICAL'</font>, <font color='green'>'ELECTRICAL - SPECIAL SYSTEMS'</font>,<font color='green'> 'ELECTRONICS'</font>, <font color='green'>'EXCAVATION'</font>, 'EXTERIOR STRUCTURES', 'FEES - CONTENTS MISC.', <font color='green'>'FENCING'</font>, <font color='green'>'FINISH CARPENTRY / TRIMWORK'</font>, 'FINISH HARDWARE', 'FIRE PROTECTION SYSTEMS', 'FIREARMS & ACCESSORIES', 'FIREPLACES', <font color='green'>'FLOOR COVERING - CARPET'</font>, <font color='green'>'FLOOR COVERING - CERAMIC TILE'</font>, <font color='green'>'FLOOR COVERING - RESILIENT'</font>, <font color='green'>'FLOOR COVERING - STONE'</font>, <font color='green'>'FLOOR COVERING - VINYL'</font>, <font color='green'>'FLOOR COVERING - WOOD'</font>, <font color='green'>'FRAMING & ROUGH CARPENTRY'</font>, <font color='green'>'FURNITURE - HOME & OFFICE'</font>, 'GENERAL DEMOLITION', <font color='green'>'GLASS, GLAZING, & STOREFRONTS'</font>, <font color='green'>'HAZARDOUS MATERIAL REMEDIATION'</font>, 'HEALTH & MEDICAL SUPPLIES', <font color='green'>'HEAT,  VENT & AIR CONDITIONING'</font>, 'HEAVY EQUIPMENT', 'HOBBIES & COLLECTIBLES', <font color='green'>'HOUSEWARES - DINING & FLATWARE'</font>, <font color='green'>'HOUSEWARES - HOME DECOR'</font>, 'INFANT & BABY RELATED GOODS', <font color='green'>'INSULATION'</font>, <font color='green'>'INSULATION - MECHANICAL'</font>, <font color='green'>'INTERIOR LATH & PLASTER'</font>, <font color='green'>'JEWELRY & WATCHES'</font>, <font color='green'>'KITCHENWARE'</font>, 'LABOR ONLY', 'LANDSCAPING', <font color='green'>'LAWN, GARDEN & PATIO'</font>, <font color='green'>'LIGHT FIXTURES'</font>, <font color='green'>'LINENS & SOFTGOODS'</font>, 'LUGGAGE, BAGS & ACCESSORIES', <font color='green'>'MARBLE - CULTURED OR NATURAL'</font>, 'MASONRY', <font color='green'>'METAL STRUCTURES & COMPONENTS'</font>, <font color='green'>'MIRRORS & SHOWER DOORS'</font>, 'MISC. EQUIPMENT - AGRICULTURAL', 'MISC. EQUIPMENT - COMMERCIAL', 'MOBILE HOMES, SKIRTING & SETUP', 'MOISTURE PROTECTION', 'MUSIC, MOVIES & MEDIA', 'MUSICAL INSTRUMENTS & EQUIP.', 'OFFICE SUPPLIES', <font color='green'>'ORNAMENTAL IRON'</font>, 'PAINTING', 'PANELING & WOOD WALL FINISHES', 'PERISHABLE - NON-PERISHABLE', 'PERMITS AND FEES', 'PERSONAL CARE & BEAUTY', 'PET & ANIMAL SUPPLIES', <font color='green'>'PLUMBING'</font>, <font color='green'>'ROOFING'</font>, <font color='green'>'SCAFFOLDING'</font>, <font color='green'>'SIDING'</font>, <font color='green'>'SOFFIT, FASCIA, & GUTTER'</font>, 'SPECIALTY ITEMS', 'SPORTING GOODS & OUTDOORS', <font color='green'>'STAIRS'</font>, <font color='green'>'STEEL COMPONENTS'</font>, <font color='green'>'STEEL JOIST COMPONENTS'</font>, <font color='green'>'STUCCO & EXTERIOR PLASTER'</font>, <font color='green'>'SWIMMING POOLS & SPAS'</font>, 'TEMPORARY REPAIRS', <font color='green'>'TILE'</font>, <font color='green'>'TIMBER FRAMING'</font>, <font color='green'>'TOILET & BATH ACCESSORIES'</font>, 'TOOLS', 'TOYS & GAMES', 'TRAUMA/CRIME SCENE REMEDIATION', 'USER DEFINED ITEMS', <font color='green'>'WALLPAPER'</font>, 'WATER EXTRACTION & REMEDIATION', <font color='green'>'WINDOW REGLAZING & REPAIR'</font>, <font color='green'>'WINDOW TREATMENT'</font>, <font color='green'>'WINDOWS - ALUMINUM'</font>, <font color='green'>'WINDOWS - SKYLIGHTS'</font>, <font color='green'>'WINDOWS - SLIDING PATIO DOORS'</font>, <font color='green'>'WINDOWS - VINYL'</font>, <font color='green'>'WINDOWS - WOOD']</font>

In [65]:
import re, os, sys
import logging

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')

logger = logging.getLogger()
handler = logging.StreamHandler(stream=sys.stdout)
logger.addHandler(handler)

data["category"] = ""
data["material"] = ""

category_items = {
    "Electrical" : ["ELECTRIC", "ELECTRONICS", "CAMERA", "COMPUTER"],
    "Construction" : ["SCAFFOLDING", "GUTTER", "INSULATION", "ROOFING", "PLASTER", "SIDING", 'STAIRS', 'EXCAVATION',
                     'FENCING', 'MOBILE HOMES', 'MASONRY', 'PAINTING', 
                      "ROOFING", "PLASTER", "WALL", 'CEILING/WALL HANGINGS', "TILE", 'STAIRS', 'GENERAL DEMOLITION',
                     'CONCRETE', 'ASPHALT', 'MARBLE'],
    "Furniture" : ["DOORS", "FURNITURE", "WINDOW", 'CABINETRY', 'PANELING & WOOD WALL FINISHES', 'CARPENTRY'],
    "Scrap metal" : ["METAL", "ORNAMENTAL IRON", 'STEEL'],
    "Sanitary equipment" : ['PLUMBING', 'TOILET & BATH ACCESSORIES', 'SWIMMING POOLS & SPAS'],
    "Paper materials" : ["PAPER", "BOOKS, MAGAZINES & PERIODICALS", 'DOCUMENTS & VALUABLE PAPERS'],
    "Home appliances & decor" : ['HEAT,  VENT & AIR CONDITIONING', 'CLEAN APPLIANCES', 'CONT: CLEAN - LAMPS OR VASES',
                              'HOME DECOR', 'HOUSEWARES', 'KITCHENWARE', 'LAWN, GARDEN & PATIO', 'LIGHT FIXTURES'],
    "Hazardous" : ['HAZARDOUS', 'TOXIC'],
    "Uphostry" : ["UPHOSTRY", "PLASTER", "VINYL", "FLOOR COVERING", "ACOUSTICAL TREATMENTS"],
    "Clothing" : ["CLOTH", "LINEN", 'GARMEN'],
    "Glass" : ["GLASS"]
} 

category_regex = {k:"|".join(v) for k,v in category_items.items()}

material_items = {
    "glass" : ["glass", "mirror", "lamp", "skylights"],
    "stone" : ["stone", "masonry", "marble"],
    "metal" : ["metal", "steel", "iron", "steel", "copper", "aluminum"],
    "paper" : ["paper", "book", "magazines", "document"],
    "wood" : ["wood", "furniture", "timber"],
    "fiber" : ["fiber"],
    "precious materials" : ["jewel", "gold", "silver"],
    "ceramic" : ["ceramics"],
    "plastic" : ["plastic", "polymers"],
    "vinyl" : ["vinyl"]
}

material_regex = {k:"|".join(v) for k,v in material_items.items()}

def add_tag(data, mask, tag_col, tag):
        """
        Aggregates multiple tags in the single value of type tag1,tag2,tag3,...
        :param data:           an input DataFrame
        :param mask:            an input DataFrame mask     
        :param tag_col:         name of the tag column
        :param tag:             an input tag to incorporate in tag column
        """
        if tag_col == "primary_tags":
            tag_mask = (data[tag_col].fillna("") != "")
            logging.warning("{}/{} items already belong to {} or other lists".format((mask & tag_mask).sum(),
                                                                    data.shape[0],
                                                                    tag
                                                                )
                        )
        else:
            tag_mask = data[tag_col].fillna("").str.contains(tag+",") | data[tag_col].fillna("").str.contains(","+tag) | (data[tag_col] == tag)
            logging.warning("{}/{} items are already tagged as {}".format((mask & tag_mask).sum(),
                                                                    data.shape[0],
                                                                    tag
                                                                )
                        )

        logging.warning("Tagged {}/{} items as {}".format((mask & (~tag_mask)).sum(),
                                                                    data.shape[0],
                                                                    tag
                                                                )
                       )
        def add_tag_to_str(x, tag):
            
            if x == "":
                return tag

            if not tag+"," in x and not ","+tag in x:
                x = x + "," + tag

            return x

        if (mask & (~tag_mask)).sum() > 0:
            data.loc[mask & (~tag_mask), tag_col] = data.loc[mask & (~tag_mask), tag_col].fillna("").apply(lambda x: add_tag_to_str(x, tag))

        return data


for category, pattern in category_regex.items():
    mask = data["subcategory"].astype(str).str.contains(pattern, flags=re.IGNORECASE)
    data.loc[mask, "category"] = category
    
for material, pattern in material_regex.items():
    mask = data["subcategory"].astype(str).str.contains(pattern, flags=re.IGNORECASE)
    data = add_tag(data, mask, "material", material)
    
data

08:38:06 WARNING:0/1357348 items are already tagged as glass


0/1357348 items are already tagged as glass


08:38:06 WARNING:Tagged 8397/1357348 items as glass


Tagged 8397/1357348 items as glass


08:38:09 WARNING:0/1357348 items are already tagged as stone


...

08:38:25 WARNING:0/1357348 items are already tagged as vinyl


0/1357348 items are already tagged as vinyl


08:38:25 WARNING:Tagged 8716/1357348 items as vinyl


Tagged 8716/1357348 items as vinyl


,subcategory,item_description,div_cd,ls_date,state,zip,reason,item_unit_cd,item_quantity,count,category,material
0,PANELING & WOOD WALL FINISHES,WATERPROOF PANELING W/TRIM - PREMIUM GRADE,PI,201910,TX,76063.0,Hail,SF,324.28,2,Furniture,wood
1,PANELING & WOOD WALL FINISHES,WATERPROOF PANELING W/TRIM - PREMIUM GRADE,BI,201609,TX,76110.0,Water (NW),SF,73.33,1,Furniture,wood
2,PANELING & WOOD WALL FINISHES,WATERPROOF PANELING W/TRIM - HIGH GRADE,PI,201605,TX,79902.0,Water (NW),SF,413.42,1,Furniture,wood
3,PANELING & WOOD WALL FINISHES,WATERPROOF PANELING W/TRIM - COMMERCIAL (FRP),BI,201910,TX,75220.0,Wind,SF,404.39,1,Furniture,wood
4,PANELING & WOOD WALL FINISHES,WATERPROOF PANELING W/TRIM - COMMERCIAL (FRP),BI,201906,TX,75206.0,Fire/Smoke,SF,277.98,1,Furniture,wood
...,...,...,...,...,...,...,...,...,...,...,...,...
1357343,WINDOWS - WOOD,"ADD ON FOR ""LOW E"" GLASS",PI,201804,TX,75032.0,Hail,SF,NaN,1,Furniture,wood
1357344,WINDOWS - WOOD,"ADD ON FOR ""LOW E"" GLASS",PI,201707,TX,79012.0,Hail,SF,57.00,1,Furniture,wood
1357345,WINDOWS - WOOD,"ADD ON FOR ""LOW E"" GLASS",PI,201706,TX,75840.0,Crime,SF,32.00,1,Furniture,wood
1357346,WINDOWS - WOOD,"ADD ON FOR ""LOW E"" GLASS",PI,201604,TX,75098.0,Hail,SF,70.50,2,Furniture,wood


In [68]:
def save_data_to_csv(data, filename, **kwargs):
        """
        Save df to csv
        :param data: df to save
        :param filename: path to file
        :param kwargs: additional arguments for DataFrame.to_csv method
        """
        dirname = os.path.dirname(filename)
        os.makedirs(dirname, exist_ok=True)
        data.to_csv(filename, **kwargs)
        
def save_data_to_excel(data, filename, **kwargs):
        """
        Save df to csv
        :param data: df to save
        :param filename: path to file
        :param kwargs: additional arguments for DataFrame.to_csv method
        """
        dirname = os.path.dirname(filename)
        os.makedirs(dirname, exist_ok=True)
        data.to_excel(filename, **kwargs)

csv_file = ("data/Pentatonic_Xact_Categories_5yr_v1.csv")
save_data_to_csv(data, csv_file, index=False)

['WINDOWS - ALUMINUM',
 'WINDOWS - SKYLIGHTS',
 'WINDOWS - SLIDING PATIO DOORS',
 'WINDOWS - VINYL',
 'WINDOWS - WOOD']